In [1]:
library(tidyverse)
library(neurobase)
library(pbapply)
library(readr)
library(lme4)
library(parallel)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: oro.nifti

oro.nifti 0.11.4


Attaching package: ‘oro.nifti’


The following object is masked from ‘package:dplyr’:

    slice


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack




In [2]:
# Define the Patient class
Patient <- setRefClass(
    "Patient",
    fields = list(
        id = "character",
        dataset = "character",
        site = "factor",
        age = "numeric",
        sex = "factor",
        diagnosis = "factor",
        ses = "character",
        img = "character",
        patient_data = "data.frame"
    ),
    methods = list(
        initialize = function(i, metadata) {
            id <<- metadata$subj_id[i]
            dataset <<- metadata$dataset[i]
            site <<- as.factor(metadata$site_string[i])
            age <<- metadata$age[i]
            sex <<- as.factor(metadata$sex_string[i])
            diagnosis <<- as.factor(metadata$diagnosis[i])
            ses <<- metadata$ses[i]
            
            if (is.na(ses)) {
                patient_dir <- file.path("/fs04/kg98/trangc/VBM/data", dataset, id, "anat")
                img <<- file.path(patient_dir, paste0("s6mwp1", id, "_T1w.nii"))
            } else {
                patient_dir <- file.path("/fs04/kg98/trangc/VBM/data", dataset, id, ses, "anat")
                img <<- file.path(patient_dir, paste0("s6mwp1", id, "_", ses, "_T1w.nii"))
            }
        },
        
        get_patient_voxels = function(atlas, nparcs = 66) {
            
            img_data <- readnii(img)
            gmv <- img_data@.Data
            
            mask <- (atlas > 0) & (atlas <= nparcs)
            gmv <- gmv[mask]
            
            gmv_flat <- as.vector(gmv)
            voxels <- setNames(gmv_flat, seq_along(gmv_flat))
            
            return(voxels)
        },
        
        make_patient_df = function(voxels) {
            n <- length(voxels)
            data <- tibble(
            MGV = as.double(voxels),
            subj_id = rep(id, n),
            voxel = as.factor(names(voxels)),
            diagnosis = rep(diagnosis, n),
            age = rep(age, n),
            sex = rep(sex, n),
            site = rep(site, n)
            )
            
            patient_data <<- data
        }
    )
)

In [3]:
# ~40 min

# Load patient metadata
metadata <- read_csv("/fs04/kg98/trangc/VBM/data/metaVBM_SCZ.csv")
metadata <- metadata %>% mutate(diagnosis = recode(diagnosis, `1` = 0, `4` = 1))

# Load atlas
s132_img <- readnii('/fs03/kg98/gchan/Atlases/Tian/Schaefer_Tian/reordered/Schaefer2018_100Parcels_7Networks_order_Tian_Subcortex_S2_MNI152NLin6Asym_1.5mm_reordered.nii.gz')
atlas <- s132_img@.Data

# Output dataframe for lme
gmv <- tibble()

# Store the dataframes in a list and combine afterwards
patient_dfs <- list()

# Set progress bar type to "timer"
pboptions(type = "timer")

patient_dfs <- pblapply(seq_len(nrow(metadata)), function(i) {
    patient <- Patient$new(i, metadata)
    
    voxels <- patient$get_patient_voxels(atlas)
    patient$make_patient_df(voxels)
    
    return(patient$patient_data)
})

gmv <- bind_rows(patient_dfs)
print(head(gmv))

Rows: 2331 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (6): subj_id, dataset, site_string, sex_string, diagnosis_string, ses
dbl (4): site, diagnosis, age, sex

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


  |++++++++++++++++++++++++++++++++++++++++++++++++++| 100% elapsed=33m 57s
# A tibble: 6 × 7
    MGV subj_id   voxel diagnosis   age sex   site      
  <dbl> <chr>     <fct> <fct>     <dbl> <fct> <fct>     
1 0.369 sub-10002 1     1            30 M     Advan_inno
2 0.362 sub-10002 2     1            30 M     Advan_inno
3 0.353 sub-10002 3     1            30 M     Advan_inno
4 0.344 sub-10002 4     1            30 M     Advan_inno
5 0.333 sub-10002 5     1            30 M     Advan_inno
6 0.320 sub-10002 6     1            30 M     Advan_inno


In [4]:
# Split into one dataframe per voxel
voxels_all_sites <- split(gmv, gmv$voxel)

# Remove sites where all values of diagnosis are 0
sites_to_remove <- gmv %>%
    group_by(site) %>%
    filter(all(diagnosis == 0)) %>%
    pull(site)

gmv <- gmv %>%
    filter(!site %in% sites_to_remove)

    
# Split into one dataframe per voxel
voxels <- split(gmv, gmv$voxel)

In [5]:
# ~ 2h

# Initialize an empty list to store the beta values
beta_values <- list()

# Calculate beta values for all data
beta_values <- pblapply(names(voxels_all_sites), function(voxel) {
    model <- lmer(MGV ~ diagnosis + age + sex + (1 | site), data = voxels_all_sites[[voxel]])
    beta_diagnosis <- summary(model)$coefficients["diagnosis0", "Estimate"]
    return(beta_diagnosis)
})

beta_values <- setNames(beta_values, names(voxels_all_sites))
# Create the initial beta_df with all data
beta_df <- data.frame(voxel = names(beta_values), diagnosis_beta = as.numeric(scale(unlist(beta_values))))

  |                                                  | 0 % ~calculating  

  |++++++                                            | 11% ~01h 27m 33s  

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.003414 (tol = 0.002, component 1)”


  |+++++++                                           | 14% ~01h 25m 30s  

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.00348354 (tol = 0.002, component 1)”
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.0033564 (tol = 0.002, component 1)”


...

  |++++++++++++++++++++++++++++++++++++++            | 75% ~25m 56s      

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.00319182 (tol = 0.002, component 1)”


  |+++++++++++++++++++++++++++++++++++++++++++       | 86% ~14m 41s      

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.00317445 (tol = 0.002, component 1)”


  |++++++++++++++++++++++++++++++++++++++++++++++++++| 100% elapsed=01h 46m 18s


In [6]:
beta_df$voxel <- as.integer(beta_df$voxel)
beta_df <- beta_df[order(beta_df$voxel), ]
write.csv(beta_df, "./results/betas_v2.csv", row.names = FALSE)
# write.table(beta_df$diagnosis_beta, "betas_diag.csv", col.names = FALSE, row.names = FALSE, sep = ",")

In [7]:
# Create a copy of the atlas to modify
lme_betas <- atlas
# Convert values > 66 in atlas to 0
lme_betas[lme_betas > 66] <- 0
# Create a mask for the values in the atlas
mask <- (lme_betas > 0)

# Flatten the mask and atlas
flat_mask <- as.vector(mask)
flat_atlas <- as.vector(lme_betas)

# Replace values in the atlas with corresponding values from beta_df
flat_atlas[flat_mask] <- beta_df$diagnosis_beta

# Reshape the modified atlas back to its original shape
lme_betas <- array(flat_atlas, dim = dim(atlas))

# Combine the modified atlas with the header from s132_img
modified_img <- s132_img
modified_img@.Data <- lme_betas

# Save the modified image to a NIfTI file
write_nifti(modified_img, "./results/lme_betas_2.nii")

In [8]:
# Create a copy of lme_betas to store the mean values
mean_lme_betas <- lme_betas

# Get unique ROI values from the atlas
rois <- unique(atlas[atlas > 0])

# Iterate over each ROI
for (roi in rois) {
    # Create a mask for the current ROI
    roi_mask <- (atlas == roi)
    
    # Extract the corresponding values from lme_betas
    roi_values <- lme_betas[roi_mask]
    
    # Calculate the mean grey value
    mean_value <- mean(roi_values, na.rm = TRUE)
    
    # Assign the mean value back to all voxels in the ROI
    mean_lme_betas[roi_mask] <- mean_value
}

# Combine the modified atlas with the header from s132_img
modified_img <- s132_img
modified_img@.Data <- mean_lme_betas

# Save the modified image to a NIfTI file
write_nifti(modified_img, "./results/mean_lme_betas_2.nii")

In [ ]:
# # Calculate beta values for each site separately
# sites <- unique(gmv$site)
# for (site in sites) {
#     beta_values <- list()

#     for (voxel in names(voxels)) {
#         df <- voxels[[voxel]] %>% filter(site == !!site)
        
#         model <- lm(MGV ~ diagnosis + age + sex, data = df)
#         model_summary <- summary(model)
#         if ("diagnosis0" %in% rownames(model_summary$coefficients)) {
#             beta_diagnosis <- model_summary$coefficients["diagnosis0", "Estimate"]
#             beta_values[[voxel]] <- beta_diagnosis
#         } else {
#             beta_values[[voxel]] <- NA
#         }
#     }
#     # Update beta_df with each site
#     beta_df[[site]] <- as.numeric(scale(unlist(beta_values)))
# }

# print(head(beta_df))

In [ ]:
# Calculate the correlation between each column and diagnosis_beta
correlations <- sapply(beta_df[-1], function(x) cor(beta_df$diagnosis_beta, x, use = "complete.obs"))

# Print the correlations
# Convert correlations to a data frame
correlation_df <- data.frame(site = names(correlations), correlation = correlations)

# Print the correlation data frame
print(correlation_df, row.names = FALSE)
write.csv(correlation_df, "correlations.csv", row.names = FALSE)
